In [129]:
import pandas as pd
import requests
from datetime import datetime

In [130]:
# Import library yang diperlukan
from flask_mysqldb import MySQL
from flask import Flask
import pandas as pd

# Inisialisasi aplikasi Flask dan koneksi MySQL
app = Flask(__name__)

# Konfigurasi database MySQL
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''  # Ubah sesuai password MySQL Anda
app.config['MYSQL_DB'] = 'penjadwalan'  # Nama database Anda

# Inisialisasi MySQL
mysql = MySQL(app)

# Membuat konteks aplikasi Flask untuk akses database
with app.app_context():
    # Query untuk mengambil data kelas
    query = """
        SELECT 
            kelas_db.id_kelas, 
            matakuliah_db.matakuliah,
            kelas_db.skala,
            kelas_db.kelas,
            GROUP_CONCAT(DISTINCT dosen_db.dosen ORDER BY kelas_dosen.urutan_dosen SEPARATOR '/n ') AS dosen,
            GROUP_CONCAT(DISTINCT rooms_db.rooms ORDER BY kelas_rooms.urutan_rooms SEPARATOR ', ') AS rooms,
            kelas_db.kapasitas,
            kelas_db.createdAt,
            matakuliah_db.jenjang,
            matakuliah_db.wp,
            GROUP_CONCAT(DISTINCT semester_db.semester ORDER BY semester_db.semester SEPARATOR ', ') AS semester_list,
            matakuliah_db.durasi
        FROM 
            kelas_db
        JOIN 
            matakuliah_db ON kelas_db.matkul_id = matakuliah_db.id_matkul
        LEFT JOIN 
            kelas_dosen ON kelas_db.id_kelas = kelas_dosen.kelas_id
        LEFT JOIN 
            dosen_db ON kelas_dosen.dosen_id = dosen_db.id_dosen
        LEFT JOIN 
            kelas_rooms ON kelas_db.id_kelas = kelas_rooms.kelas_id
        LEFT JOIN 
            rooms_db ON kelas_rooms.rooms_id = rooms_db.id_rooms
        LEFT JOIN 
            semester_matakuliah ON matakuliah_db.id_matkul = semester_matakuliah.matkul_id
        LEFT JOIN 
            semester_db ON semester_matakuliah.semester_id = semester_db.id_semester
        GROUP BY 
            kelas_db.id_kelas, 
            matakuliah_db.matakuliah, 
            kelas_db.skala, 
            kelas_db.kelas, 
            kelas_db.kapasitas, 
            kelas_db.createdAt, 
            matakuliah_db.jenjang,
            matakuliah_db.wp,
            matakuliah_db.durasi
    """

    # Menjalankan query dan mendapatkan data
    cur = mysql.connection.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    cur.close()

    # Memproses data yang diambil dan membuat DataFrame
    kelas_data = [
        {
            'id_kelas': row[0],
            'matakuliah': row[1],
            'skala': row[2],
            'kelas': row[3],
            'dosen': row[4].split('/n ') if row[4] else [],
            'rooms': (row[5].split(', ') if row[5] else []) + ['alternatif'],
            'kapasitas': row[6],
            'createdAt': row[7],
            'jenjang': row[8],
            'wp': row[9],
            'semester': [int(x) for x in row[10].split(', ')] if row[10] else [],
            'durasi': row[11],
        }
        for row in rows
    ]

    # Membuat DataFrame untuk menampilkan data
    df = pd.DataFrame(kelas_data)

    # Menampilkan DataFrame
    # display(df)


In [131]:
# Fungsi untuk mengecek apakah dua kelas memiliki dosen yang sama
def are_dosen_same(dosen1, dosen2):
    return set(dosen1) == set(dosen2)

# Fungsi untuk melakukan pengecekan dan penggabungan
def merge_classes(df):
    merged_classes = []  # Menyimpan nama kelas yang sudah digabung
    merged_pairs = set()  # Untuk menyimpan pasangan kelas yang sudah digabung
    class_mapping = {}  # Menyimpan nama kelas yang sudah digabungkan

    for i, row1 in df.iterrows():
        for j, row2 in df.iterrows():
            # Menghindari membandingkan kelas dengan dirinya sendiri atau duplikasi
            if i >= j:
                continue
                
            # Pengecekan 1: Matakuliah sama
            if row1['matakuliah'] == row2['matakuliah']:
                # Pengecekan 2: Dosen sama (periksa semua dosen di kelas)
                if are_dosen_same(row1['dosen'], row2['dosen']):
                    # Pengecekan 3: Skala sama
                    if row1['skala'] == row2['skala']:
                        # Membuat nama gabungan kelas
                        kelas1 = row1['kelas']
                        kelas2 = row2['kelas']
                        
                        # Memastikan bahwa kelas belum digabung
                        if kelas1 not in class_mapping and kelas2 not in class_mapping:
                            # Gabungkan nama kelas
                            new_kelas = f"{row1['matakuliah']} {kelas1.split('.')[-1]} & {kelas2.split('.')[-1]}"
                            merged_classes.append(new_kelas)
                            
                            # Gabungkan rooms dari salah satu kelas
                            merged_rooms = row1['rooms']  # Ambil rooms dari kelas pertama

                            # Tandai kelas yang sudah digabung
                            class_mapping[kelas1] = {
                                'kelas_gabung': new_kelas,
                                'rooms': merged_rooms
                            }
                            class_mapping[kelas2] = {
                                'kelas_gabung': new_kelas,
                                'rooms': merged_rooms
                            }
    return merged_classes, class_mapping

# Panggil fungsi untuk menggabungkan kelas
merged_classes, class_mapping = merge_classes(df)

# Membuat DataFrame hasil penggabungan
merged_data = []  # Data yang sudah digabung
non_merged_data = []  # Data yang tidak digabung

for index, row in df.iterrows():
    kelas = row['kelas']
    # Menambahkan kelas gabungan jika ada
    if kelas in class_mapping:
        kelas_gabung = class_mapping[kelas]['kelas_gabung']
        rooms_gabung = class_mapping[kelas]['rooms']
    else:
        kelas_gabung = kelas
        rooms_gabung = row['rooms']
    
    # Menambahkan baris baru jika kelas telah digabungkan
    if kelas_gabung != kelas:
        # Pastikan hanya menambahkan kelas yang digabungkan sekali
        if kelas_gabung not in [item['kelas'] for item in merged_data]:
            merged_data.append({
                'matakuliah': row['matakuliah'],
                'kelas': kelas_gabung,
                'jenjang': row['jenjang'],
                'skala': row['skala'],
                'wp': row['wp'],
                'semester': row['semester'],
                'dosen': row['dosen'],
                'kapasitas': row['kapasitas'],
                'durasi': row['durasi'],
                'rooms': rooms_gabung
            })
    else:
        # Jika kelas tidak digabung, masukkan ke non_merged_data
        non_merged_data.append({
            'matakuliah': row['matakuliah'],
            'kelas': kelas,
            'jenjang': row['jenjang'],
            'skala': row['skala'],
            'wp': row['wp'],
            'semester': row['semester'],
            'dosen': row['dosen'],
            'kapasitas': row['kapasitas'],
            'durasi': row['durasi'],
            'rooms': rooms_gabung
        })

# Membuat DataFrame baru dari data yang telah digabungkan dan yang tidak digabungkan
df_merged = pd.DataFrame(merged_data)
df_non_merged = pd.DataFrame(non_merged_data)

# Menampilkan data yang telah digabungkan
if not df_merged.empty:
    print("Data telah berhasil digabungkan:")
else:
    print("Tidak ada data yang digabungkan.")


Data telah berhasil digabungkan:


In [132]:
kelas_data = merged_data + non_merged_data
before_kelas_data = merged_data*2 + non_merged_data

df_kelas = pd.DataFrame(kelas_data)
df_before_kelas = pd.DataFrame(before_kelas_data)

# Menampilkan kelas_data
if not df_kelas.empty:
    # display(df_kelas)
    print(f"Jumlah data sebelum digabung: {df_before_kelas.shape[0]}")
    print(f"Jumlah data setelah digabung: {df_kelas.shape[0]}")
else:
    print("Tidak ada data untuk ditampilkan.")

Jumlah data sebelum digabung: 127
Jumlah data setelah digabung: 118


In [133]:
# Import library yang diperlukan
from flask_mysqldb import MySQL
from flask import Flask
import pandas as pd

# Inisialisasi aplikasi Flask dan koneksi MySQL
app = Flask(__name__)

# Konfigurasi database MySQL
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = ''  # Ubah sesuai password MySQL Anda
app.config['MYSQL_DB'] = 'penjadwalan'  # Nama database Anda

# Inisialisasi MySQL
mysql = MySQL(app)

# Membuat konteks aplikasi Flask untuk akses database
with app.app_context():
    # Query untuk mengambil data ruangan
    query = """
        SELECT 
            ruangan_db.id_ruangan, 
            ruangan_db.name, 
            ruangan_db.kapasitas, 
            GROUP_CONCAT(jenis_db.jenis ORDER BY jenis_ruangan.jenis_id SEPARATOR ', ') AS jenis_list,
            ruangan_db.createdAt
        FROM 
            ruangan_db
        LEFT JOIN 
            jenis_ruangan ON ruangan_db.id_ruangan = jenis_ruangan.ruangan_id
        LEFT JOIN 
            jenis_db ON jenis_ruangan.jenis_id = jenis_db.id_jenis
        GROUP BY 
            ruangan_db.id_ruangan, ruangan_db.name, ruangan_db.kapasitas, ruangan_db.createdAt
    """

    # Menjalankan query dan mendapatkan data
    cur = mysql.connection.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    cur.close()

    # Memproses data yang diambil dan membuat DataFrame
    ruangan_data = [
        {
            'id_ruangan': row[0],
            'name': row[1],
            'kapasitas': row[2],
            'jenis': row[3].split(', ') if row[3] else [],  # Memecah jenis ruangan yang dipisahkan koma
            'createdAt': row[4],
        }
        for row in rows
    ]

    # Membuat DataFrame untuk menampilkan data
    df = pd.DataFrame(ruangan_data)


In [134]:
hari_data = [
    "Senin", 
    "Selasa", 
    "Rabu", 
    "Kamis", 
    "Jumat"
]
jam_data = [
    "07:50 - 08:40", "08:40 - 09:30",
    "09:30 - 10:20", "10:20 - 11:10", 
    "11:10 - 12:00", "12:00 - 13:00",
    "13:00 - 13:50", "13:50 - 14:40", 
    "14:40 - 15:30", "15:30 - 16:20", 
    "16:20 - 17:10"
]

In [135]:
jadwal_s1 = []
scheduled_classes = []  
tabrakan_data = []
failed_classes = []

def are_slots_connected(jam_slots):
    # Periksa apakah slot mengandung jam istirahat
    if "12:00 - 13:00" in jam_slots:
        return False

    # Periksa konektivitas antar slot
    for i in range(len(jam_slots) - 1):
        if jam_slots[i] == "11:10 - 12:00" and jam_slots[i + 1] == "13:00 - 13:50":
            return False

    return True

# Fungsi untuk mengecek tabrakan jadwal dengan rentang minggu
def is_schedule_valid(kelas, dosen, ruangan, hari, jam_slots, minggu):
    # Definisikan rentang minggu dalam format numerik untuk perbandingan
    minggu_map = {
        "1-6": (1, 6),
        "7-11": (7, 11),
        "12-16": (12, 16),
        "1-8": (1, 8),
        "9-16": (9, 16),
        "1-16": (1, 16)
    }

    start_minggu, end_minggu = minggu_map[minggu]

    # Periksa apakah ada tabrakan dengan jadwal lain yang sudah terpasang
    for entry in jadwal_s1:
        entry_start_minggu, entry_end_minggu = minggu_map[entry['minggu']]

        # Cek apakah ada tabrakan minggu dan jadwal (ruangan atau dosen)
        if entry['ruangan'] == ruangan and entry['hari'] == hari and \
        entry_start_minggu <= end_minggu and entry_end_minggu >= start_minggu and \
        any(jam in entry['jam'] for jam in jam_slots):
            return False

        if entry['dosen'] == dosen and entry['hari'] == hari and \
        entry_start_minggu <= end_minggu and entry_end_minggu >= start_minggu and \
        any(jam in entry['jam'] for jam in jam_slots):
            return False
    return True

def check_time_overlap(jam_terjadwal, jam_baru):
    """Memeriksa apakah dua jam tumpang tindih"""
    # Jika jam_terjadwal adalah list, periksa tiap elemen dalam list
    if isinstance(jam_terjadwal, list):
        for jam in jam_terjadwal:
            if check_time_overlap(jam, jam_baru):  # Rekursif untuk tiap slot waktu
                return True
        return False

    # Ambil waktu mulai dan selesai dari kedua jam
    start_existing, end_existing = jam_terjadwal.split(" - ")
    start_new, end_new = jam_baru.split(" - ")

    # Ganti titik (.) dengan tanda titik dua (:) untuk memastikan format waktu yang valid
    start_existing = start_existing.replace('.', ':')
    end_existing = end_existing.replace('.', ':')
    start_new = start_new.replace('.', ':')
    end_new = end_new.replace('.', ':')

    # Konversi ke format datetime untuk perbandingan
    start_existing = datetime.strptime(start_existing, "%H:%M")
    end_existing = datetime.strptime(end_existing, "%H:%M")
    start_new = datetime.strptime(start_new, "%H:%M")
    end_new = datetime.strptime(end_new, "%H:%M")

    # Periksa apakah ada tumpang tindih
    if (start_new < end_existing) and (start_existing < end_new):
        return True
    return False

def is_odd(number):
    return number % 2 != 0

In [136]:
# filter S1, Nasional, semester ganjil
def filter_and_sort_kelas(kelas_data, w_p, jenjang, skala, target_ganjil=True):
    # Filter kelas berdasarkan kondisi yang diberikan
    filtered_kelas = [
        kelas for kelas in kelas_data
        if kelas.get("wp") == w_p and
        kelas.get("jenjang") == jenjang and
        kelas.get("skala") == skala and
        (
            # Jika target adalah ganjil, cek apakah ada angka ganjil di semester
            (target_ganjil and any(is_odd(s) for s in kelas.get("semester", []))) or
            # Jika target adalah genap, cek apakah ada angka genap di semester
            (not target_ganjil and any(not is_odd(s) for s in kelas.get("semester", [])))
        )
    ]

    # Urutkan berdasarkan kapasitas dan durasi
    return sorted(
        filtered_kelas,
        key=lambda x: (x["kapasitas"], x["durasi"]),
        reverse=True
    )
filtered_kelas_w = filter_and_sort_kelas(kelas_data, "W", "S1", "Nasional", target_ganjil=True)  # Untuk semester ganjil
filtered_kelas_p = filter_and_sort_kelas(kelas_data, "P", "S1", "Nasional", target_ganjil=True)  # Untuk semester genap
filtered_kelas_data = filtered_kelas_w + filtered_kelas_p

total_filtered_kelas = len(filtered_kelas_data)
print(f"Data S1 Nasional (Ganjil) : {total_filtered_kelas}")

def filter_ruangan(kelas, ruangan_data):
    # List prioritas rooms dari kelas
    rooms_prioritas = kelas.get('rooms', [])
    kapasitas_kelas = kelas['kapasitas']

    # Inisialisasi ruangan terpilih kosong
    ruangan_terpilih = []

    # Iterasi berdasarkan prioritas room
    for room_prioritas in rooms_prioritas:
        # Filter ruangan berdasarkan jenis dan kapasitas
        ruangan_cocok = [
            ruangan for ruangan in ruangan_data
            if room_prioritas in ruangan.get('jenis', []) and ruangan['kapasitas'] >= kapasitas_kelas
        ]

        # Jika ditemukan ruangan cocok, tambahkan ke ruangan terpilih dan keluar dari loop
        if ruangan_cocok:
            ruangan_terpilih.extend(ruangan_cocok)
            break

    # Jika tidak ada ruangan sesuai prioritas pertama, gunakan alternatif
    if not ruangan_terpilih:
        ruangan_terpilih = [
            ruangan for ruangan in ruangan_data
            if 'alternatif' in ruangan.get('jenis', []) and ruangan['kapasitas'] >= kapasitas_kelas
        ]

    # Urutkan ruangan berdasarkan kapasitas (ascending)
    ruangan_terpilih = sorted(ruangan_terpilih, key=lambda x: x['kapasitas'])

    return ruangan_terpilih

# from datetime import datetime

excluded_day = "Rabu"
filtered_hari_data = [hari for hari in hari_data if hari != excluded_day]

def schedule_nasional_s1_ganjil_classes():
    for kelas in filtered_kelas_data:
        if 'ruangan' in kelas and 'jam' in kelas:
            continue

        is_scheduled = False
        for hari in filtered_hari_data:
            if is_scheduled:  # Stop pencarian jika sudah dijadwalkan
                break

            # Filter ruangan yang memungkinkan
            ruangan_terpilih = filter_ruangan(kelas, ruangan_data)
            if not ruangan_terpilih:
                continue

            for ruangan in ruangan_terpilih:
                if is_scheduled:
                    break

                jam_index = 0
                while jam_index <= len(jam_data) - kelas["durasi"]:
                    jam_slots = jam_data[jam_index:jam_index + kelas["durasi"]]

                    # Gabungkan validasi konektivitas dan jam istirahat
                    if not are_slots_connected(jam_slots):
                        jam_index += 1
                        continue

                    # Pembagian minggu berdasarkan jumlah dosen
                    dosen_minggu_map = []
                    if len(kelas["dosen"]) == 1:
                        dosen_minggu_map = [(kelas["dosen"][0], "1-16")]
                    elif len(kelas["dosen"]) == 2:
                        dosen_minggu_map = [
                            (kelas["dosen"][0], "1-8"),
                            (kelas["dosen"][1], "9-16")
                        ]
                    elif len(kelas["dosen"]) == 3:
                        dosen_minggu_map = [
                            (kelas["dosen"][0], "1-6"),
                            (kelas["dosen"][1], "7-11"),
                            (kelas["dosen"][2], "12-16")
                        ]

                    conflict_found = False
                    for dosen, minggu in dosen_minggu_map:
                        # Check if the schedule is valid for the current instructor and week range
                        if not is_schedule_valid(kelas, dosen, ruangan["name"], hari, jam_slots, minggu):
                            conflict_found = True
                            break
                            
                        if kelas["wp"] == 'W':
                            for jadwal in scheduled_classes:
                                if jadwal["hari"] == hari:
                                    for jam_slot in jam_slots:
                                        if check_time_overlap(jadwal["jam"], jam_slot):
                                            if jadwal["semester"] == kelas["semester"]:
                                                # Cek apakah matakuliah berbeda
                                                if jadwal["matakuliah"] != kelas["matakuliah"]:
                                                    conflict_found = True
                                                    break
                                    if conflict_found:
                                        break
                            
                    if conflict_found:
                        jam_index += 1
                        continue

                    # Jadwalkan kelas jika valid
                    for dosen, minggu in dosen_minggu_map:
                        jadwal_s1.append({
                            "kelas": kelas["kelas"],
                            "matakuliah": kelas["matakuliah"],
                            "dosen": dosen,
                            "ruangan": ruangan["name"],
                            "w/p": kelas["wp"],
                            "semester": kelas["semester"],
                            "hari": hari,
                            "jam": jam_slots,
                            "minggu": minggu,
                            "skala": kelas["skala"]
                        })

                    # Tandai kelas sebagai dijadwalkan
                    kelas['ruangan'], kelas['jam'], kelas['hari'] = ruangan, jam_slots, hari
                    is_scheduled = True
                    scheduled_classes.append(kelas)
                    break

        if not is_scheduled:
            failed_classes.append(kelas)
            print(f">>>>> Warning: Kelas {kelas['kelas']} tidak dapat dijadwalkan di ruangan dengan room: {kelas['rooms']}")

# Menjalankan penjadwalan
schedule_nasional_s1_ganjil_classes()

Data S1 Nasional (Ganjil) : 91


In [137]:
# filter S1, MBKM, semester ganjil
def filter_and_sort_kelas(kelas_data, w_p, jenjang, skala, target_ganjil=True):
    # Filter kelas berdasarkan kondisi yang diberikan
    filtered_kelas = [
        kelas for kelas in kelas_data
        if kelas.get("wp") == w_p and
        kelas.get("jenjang") == jenjang and
        kelas.get("skala") == skala and
        (
            # Jika target adalah ganjil, cek apakah ada angka ganjil di semester
            (target_ganjil and any(is_odd(s) for s in kelas.get("semester", []))) or
            # Jika target adalah genap, cek apakah ada angka genap di semester
            (not target_ganjil and any(not is_odd(s) for s in kelas.get("semester", [])))
        )
    ]

    # Urutkan berdasarkan kapasitas dan durasi
    return sorted(
        filtered_kelas,
        key=lambda x: (x["kapasitas"], x["durasi"]),
        reverse=True
    )
filtered_kelas_mbkm_w = filter_and_sort_kelas(kelas_data, "W", "S1", "MBKM", target_ganjil=True)
filtered_kelas_mbkm_p = filter_and_sort_kelas(kelas_data, "P", "S1", "MBKM", target_ganjil=True)
filtered_kelas_mbkm_data = filtered_kelas_mbkm_w + filtered_kelas_mbkm_p

total_filtered_mbkm_kelas = len(filtered_kelas_mbkm_data)
print(f"Data S1 mbkm (Ganjil) : {total_filtered_mbkm_kelas}")

def filter_ruangan(kelas, ruangan_data):
    # List prioritas rooms dari kelas
    rooms_prioritas = kelas.get('rooms', [])
    kapasitas_kelas = kelas['kapasitas']

    # Inisialisasi ruangan terpilih kosong
    ruangan_terpilih = []

    # Iterasi berdasarkan prioritas room
    for room_prioritas in rooms_prioritas:
        ruangan_cocok = [
            ruangan for ruangan in ruangan_data
            if room_prioritas in ruangan.get('jenis', []) and ruangan['kapasitas'] >= kapasitas_kelas
        ]

        # Jika ditemukan ruangan cocok, tambahkan ke ruangan terpilih dan keluar dari loop
        if ruangan_cocok:
            ruangan_terpilih.extend(ruangan_cocok)
            break

    # Jika tidak ada ruangan sesuai prioritas pertama, gunakan alternatif
    if not ruangan_terpilih:
        ruangan_terpilih = [
            ruangan for ruangan in ruangan_data
            if 'alternatif' in ruangan.get('jenis', []) and ruangan['kapasitas'] >= kapasitas_kelas
        ]

    # Urutkan ruangan berdasarkan kapasitas (ascending)
    ruangan_terpilih = sorted(ruangan_terpilih, key=lambda x: x['kapasitas'])

    return ruangan_terpilih

prioritas_day = "Rabu"
filtered_hari_data = [prioritas_day] + [hari for hari in hari_data if hari != prioritas_day]

def schedule_mbkm_s1_ganjil_classes():
    for kelas in filtered_kelas_mbkm_data:
        if 'ruangan' in kelas and 'jam' in kelas:
            continue

        is_scheduled = False
        for hari in filtered_hari_data:
            if is_scheduled:
                break

            # Filter ruangan yang memungkinkan
            ruangan_terpilih = filter_ruangan(kelas, ruangan_data)
            if not ruangan_terpilih:
                continue

            for ruangan in ruangan_terpilih:
                if is_scheduled:
                    break

                jam_index = 0
                while jam_index <= len(jam_data) - kelas["durasi"]:
                    jam_slots = jam_data[jam_index:jam_index + kelas["durasi"]]

                    # Gabungkan validasi konektivitas dan jam istirahat
                    if not are_slots_connected(jam_slots):
                        jam_index += 1
                        continue

                    # Pembagian minggu berdasarkan jumlah dosen
                    dosen_minggu_map = []
                    if len(kelas["dosen"]) == 1:
                        dosen_minggu_map = [(kelas["dosen"][0], "1-16")]
                    elif len(kelas["dosen"]) == 2:
                        dosen_minggu_map = [
                            (kelas["dosen"][0], "1-8"),
                            (kelas["dosen"][1], "9-16")
                        ]
                    elif len(kelas["dosen"]) == 3:
                        dosen_minggu_map = [
                            (kelas["dosen"][0], "1-6"),
                            (kelas["dosen"][1], "7-11"),
                            (kelas["dosen"][2], "12-16")
                        ]

                    # Validasi setiap pasangan dosen dan minggu
                    conflict_found = False
                    for dosen, minggu in dosen_minggu_map:
                        if not is_schedule_valid(kelas, dosen, ruangan["name"], hari, jam_slots, minggu):
                            conflict_found = True
                            break

                    if conflict_found:
                        jam_index += 1
                        continue

                    # Jadwalkan kelas jika valid
                    for dosen, minggu in dosen_minggu_map:
                        jadwal_s1.append({
                            "kelas": kelas["kelas"],
                            "matakuliah": kelas["matakuliah"],
                            "dosen": dosen,
                            "ruangan": ruangan["name"],
                            "w/p": kelas["wp"],
                            "semester": kelas["semester"],
                            "hari": hari,
                            "jam": jam_slots,
                            "minggu": minggu,
                            "skala": kelas["skala"]
                        })

                    # Tandai kelas sebagai dijadwalkan
                    kelas['ruangan'], kelas['jam'], kelas['hari'] = ruangan, jam_slots, hari
                    is_scheduled = True
                    scheduled_classes.append(kelas)
                    break

        if not is_scheduled:
            failed_classes.append(kelas)
            print(f">>>>> Warning: Kelas {kelas['kelas']} tidak dapat dijadwalkan di ruangan dengan room: {kelas['rooms']}")

# Menjalankan penjadwalan
schedule_mbkm_s1_ganjil_classes()

Data S1 mbkm (Ganjil) : 2


In [138]:
# filter S1, Inter, semester ganjil
def filter_and_sort_kelas(kelas_data, w_p, jenjang, skala, target_ganjil=True):
    # Filter kelas berdasarkan kondisi yang diberikan
    filtered_kelas = [
        kelas for kelas in kelas_data
        if kelas.get("wp") == w_p and
        kelas.get("jenjang") == jenjang and
        kelas.get("skala") == skala and
        (
            # Jika target adalah ganjil, cek apakah ada angka ganjil di semester
            (target_ganjil and any(is_odd(s) for s in kelas.get("semester", []))) or
            # Jika target adalah genap, cek apakah ada angka genap di semester
            (not target_ganjil and any(not is_odd(s) for s in kelas.get("semester", [])))
        )
    ]

    # Urutkan berdasarkan kapasitas dan durasi
    return sorted(
        filtered_kelas,
        key=lambda x: (x["kapasitas"], x["durasi"]),
        reverse=True
    )
filtered_kelas_inter_w = filter_and_sort_kelas(kelas_data, "W", "S1", "Inter", target_ganjil=True)
filtered_kelas_inter_p = filter_and_sort_kelas(kelas_data, "P", "S1", "Inter", target_ganjil=True)
filtered_kelas_inter_data = filtered_kelas_inter_w + filtered_kelas_inter_p

total_filtered_inter_kelas = len(filtered_kelas_inter_data)
print(f"Data S1 inter (Ganjil) : {total_filtered_inter_kelas}")

def filter_ruangan(kelas, ruangan_data):
    # List prioritas rooms dari kelas
    rooms_prioritas = kelas.get('rooms', [])
    kapasitas_kelas = kelas['kapasitas']

    # Inisialisasi ruangan terpilih kosong
    ruangan_terpilih = []

    # Iterasi berdasarkan prioritas room
    for room_prioritas in rooms_prioritas:
        # Filter ruangan berdasarkan jenis dan kapasitas
        ruangan_cocok = [
            ruangan for ruangan in ruangan_data
            if room_prioritas in ruangan.get('jenis', []) and ruangan['kapasitas'] >= kapasitas_kelas
        ]

        # Jika ditemukan ruangan cocok, tambahkan ke ruangan terpilih dan keluar dari loop
        if ruangan_cocok:
            ruangan_terpilih.extend(ruangan_cocok)
            break

    # Jika tidak ada ruangan sesuai prioritas pertama, gunakan alternatif
    if not ruangan_terpilih:
        ruangan_terpilih = [
            ruangan for ruangan in ruangan_data
            if 'alternatif' in ruangan.get('jenis', []) and ruangan['kapasitas'] >= kapasitas_kelas
        ]

    # Urutkan ruangan berdasarkan kapasitas (ascending)
    ruangan_terpilih = sorted(ruangan_terpilih, key=lambda x: x['kapasitas'])

    return ruangan_terpilih

no_prioritas_day = "Rabu"
filtered_hari_data = [hari for hari in hari_data if hari != prioritas_day] +  [no_prioritas_day]

def schedule_inter_s1_ganjil_classes():
    for kelas in filtered_kelas_inter_data:
        if 'ruangan' in kelas and 'jam' in kelas:
            continue

        is_scheduled = False
        for hari in hari_data:
            if is_scheduled:
                break

            # Filter ruangan yang memungkinkan
            ruangan_terpilih = filter_ruangan(kelas, ruangan_data)
            if not ruangan_terpilih:
                continue

            for ruangan in ruangan_terpilih:
                if is_scheduled:
                    break

                jam_index = 0
                while jam_index <= len(jam_data) - kelas["durasi"]:
                    jam_slots = jam_data[jam_index:jam_index + kelas["durasi"]]

                    # Gabungkan validasi konektivitas dan jam istirahat
                    if not are_slots_connected(jam_slots):
                        jam_index += 1
                        continue

                    # Pembagian minggu berdasarkan jumlah dosen
                    dosen_minggu_map = []
                    if len(kelas["dosen"]) == 1:
                        dosen_minggu_map = [(kelas["dosen"][0], "1-16")]
                    elif len(kelas["dosen"]) == 2:
                        dosen_minggu_map = [
                            (kelas["dosen"][0], "1-8"),
                            (kelas["dosen"][1], "9-16")
                        ]
                    elif len(kelas["dosen"]) == 3:
                        dosen_minggu_map = [
                            (kelas["dosen"][0], "1-6"),
                            (kelas["dosen"][1], "7-11"),
                            (kelas["dosen"][2], "12-16")
                        ]

                    # Validasi setiap pasangan dosen dan minggu
                    conflict_found = False
                    for dosen, minggu in dosen_minggu_map:
                        # Check if the schedule is valid for the current instructor and week range
                        if not is_schedule_valid(kelas, dosen, ruangan["name"], hari, jam_slots, minggu):
                            conflict_found = True
                            break
                            
                        if kelas["wp"] == 'W':
                            for jadwal in scheduled_classes:
                                if jadwal["skala"] == kelas["skala"]:
                                    if jadwal["hari"] == hari:
                                        for jam_slot in jam_slots:
                                            if check_time_overlap(jadwal["jam"], jam_slot):
                                                if jadwal["semester"] == kelas["semester"]:
                                                    # Cek apakah mata kuliah berbeda
                                                    if jadwal["matakuliah"] != kelas["matakuliah"]:
                                                        conflict_found = True
                                                        break
                                        if conflict_found:
                                            break
                            if conflict_found:
                                continue
                            
                    if conflict_found:
                        jam_index += 1
                        continue

                    # Jadwalkan kelas jika valid
                    for dosen, minggu in dosen_minggu_map:
                        jadwal_s1.append({
                            "kelas": kelas["kelas"],
                            "matakuliah": kelas["matakuliah"],
                            "dosen": dosen,
                            "ruangan": ruangan["name"],
                            "w/p": kelas["wp"],
                            "semester": kelas["semester"],
                            "hari": hari,
                            "jam": jam_slots,
                            "minggu": minggu,
                            "skala": kelas["skala"]
                        })

                    # Tandai kelas sebagai dijadwalkan
                    kelas['ruangan'], kelas['jam'], kelas['hari'] = ruangan, jam_slots, hari
                    is_scheduled = True
                    scheduled_classes.append(kelas)
                    break

        if not is_scheduled:
            failed_classes.append(kelas)
            print(f">>>>> Warning: Kelas {kelas['kelas']} tidak dapat dijadwalkan di ruangan dengan room: {kelas['rooms']}")

# Menjalankan penjadwalan
schedule_inter_s1_ganjil_classes()

Data S1 inter (Ganjil) : 25


In [139]:
# Mengecek kelas yang tidak terjadwalkan
def check_unscheduled_classes(filtered_kelas_inter_data, jadwal_s1):    
    # Menghitung jumlah kelas yang berbeda pada jadwal_s1
    unique_classes_in_jadwal = set(kelas['kelas'] for kelas in jadwal_s1)

    # Menampilkan hasil
    print(f"Jumlah kelas yang aktif: {len(unique_classes_in_jadwal)}")
    
    scheduled_classes_set = set([kelas['kelas'] for kelas in jadwal_s1])  # Set kelas yang sudah dijadwalkan
    unscheduled_classes = []

    for kelas in filtered_kelas_inter_data:
        if kelas['kelas'] not in scheduled_classes_set:
            unscheduled_classes.append(kelas)

    return unscheduled_classes

# Menjalankan pengecekan kelas yang tidak terjadwalkan
unscheduled_classes = check_unscheduled_classes(filtered_kelas_inter_data, jadwal_s1)

# Menampilkan hasil
if unscheduled_classes:
    print(f">>>>> Kelas yang tidak terjadwalkan: {len(unscheduled_classes)}")
    for kelas in unscheduled_classes:
        print(f"Kelas {kelas['kelas']} tidak terjadwalkan.")
else:
    print("Semua kelas sudah terjadwalkan.")


Jumlah kelas yang aktif: 118
Semua kelas sudah terjadwalkan.
